## Minimax with alpha-beta pruning

In [117]:
import random
import numpy as np
from game import Game, Move, Player
from copy import deepcopy
from tqdm import tqdm

Dummy game class to leave the real Game class untouched

In [118]:
class Dummy_Game(object):
    def __init__(self) -> None:
        self._board = np.ones((5, 5), dtype=np.uint8) * -1
        self.current_player_idx = 1

    def get_board(self): return self._board

    def single_move(self, board, from_pos, move, player_id):
        self._board = deepcopy(board)
        self.current_player_idx = player_id
        ok = self.__move(from_pos, move, player_id)
        return deepcopy(self._board), ok
    
    def check_winner_board(self, board):
        self._board = board
        return self.check_winner()

    def check_winner(self) -> int:
        for x in range(self._board.shape[0]):
            if self._board[x, 0] != -1 and all(self._board[x, :] == self._board[x, 0]): return self._board[x, 0]
        for y in range(self._board.shape[1]):
            if self._board[0, y] != -1 and all(self._board[:, y] == self._board[0, y]): return self._board[0, y]
        if self._board[0, 0] != -1 and all([self._board[x, x] for x in range(self._board.shape[0])] == self._board[0, 0]): return self._board[0, 0]
        if self._board[0, -1] != -1 and all([self._board[x, -(x + 1)] for x in range(self._board.shape[0])] == self._board[0, -1]): return self._board[0, -1]
        return -1

    def __move(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        if player_id > 2: return False
        prev_value = deepcopy(self._board[(from_pos[1], from_pos[0])])
        acceptable = self.__take((from_pos[1], from_pos[0]), player_id)
        if acceptable:
            acceptable = self.__slide((from_pos[1], from_pos[0]), slide)
            if not acceptable: self._board[(from_pos[1], from_pos[0])] = deepcopy(prev_value)
        return acceptable

    def __take(self, from_pos: tuple[int, int], player_id: int) -> bool:
        acceptable: bool = ((from_pos[0] == 0 and from_pos[1] < 5) or (from_pos[0] == 4 and from_pos[1] < 5) or (from_pos[1] == 0 and from_pos[0] < 5) or (from_pos[1] == 4 and from_pos[0] < 5)) and (self._board[from_pos] < 0 or self._board[from_pos] == player_id)
        if acceptable: self._board[from_pos] = player_id
        return acceptable

    def __slide(self, from_pos: tuple[int, int], slide: Move) -> bool:
        SIDES = [(0, 0), (0, 4), (4, 0), (4, 4)]
        if from_pos not in SIDES:
            acceptable_top: bool = from_pos[0] == 0 and (slide == Move.BOTTOM or slide == Move.LEFT or slide == Move.RIGHT)
            acceptable_bottom: bool = from_pos[0] == 4 and (slide == Move.TOP or slide == Move.LEFT or slide == Move.RIGHT)
            acceptable_left: bool = from_pos[1] == 0 and (slide == Move.BOTTOM or slide == Move.TOP or slide == Move.RIGHT)
            acceptable_right: bool = from_pos[1] == 4 and (slide == Move.BOTTOM or slide == Move.TOP or slide == Move.LEFT)
        else:
            acceptable_top: bool = from_pos == (0, 0) and (slide == Move.BOTTOM or slide == Move.RIGHT)
            acceptable_left: bool = from_pos == (4, 0) and (slide == Move.TOP or slide == Move.RIGHT)
            acceptable_right: bool = from_pos == (0, 4) and (slide == Move.BOTTOM or slide == Move.LEFT)
            acceptable_bottom: bool = from_pos == (4, 4) and (slide == Move.TOP or slide == Move.LEFT)
        acceptable: bool = acceptable_top or acceptable_bottom or acceptable_left or acceptable_right
        if acceptable:
            piece = self._board[from_pos]
            if slide == Move.LEFT:
                for i in range(from_pos[1], 0, -1): self._board[(from_pos[0], i)] = self._board[(from_pos[0], i - 1)]
                self._board[(from_pos[0], 0)] = piece
            elif slide == Move.RIGHT:
                for i in range(from_pos[1], self._board.shape[1] - 1, 1): self._board[(from_pos[0], i)] = self._board[(from_pos[0], i + 1)]
                self._board[(from_pos[0], self._board.shape[1] - 1)] = piece
            elif slide == Move.TOP:
                for i in range(from_pos[0], 0, -1): self._board[(i, from_pos[1])] = self._board[(i - 1, from_pos[1])]
                self._board[(0, from_pos[1])] = piece
            elif slide == Move.BOTTOM:
                for i in range(from_pos[0], self._board.shape[0] - 1, 1): self._board[(i, from_pos[1])] = self._board[(i + 1, from_pos[1])]
                self._board[(self._board.shape[0] - 1, from_pos[1])] = piece
        return acceptable

Computing all legal moves (when the board is empty)

In [119]:
border = []
for i in range(5):
    for j in range(5):
        if i == 0 or i == 4 or j == 0 or j == 4:
            border.append((i, j))
BORDER = (list(set(border)))
print(len(BORDER))

def tile_to_moves(tile):
    possible_moves = [Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT]
        
    if tile[0] == 0: possible_moves.remove(Move.LEFT)
    if tile[0] == 4: possible_moves.remove(Move.RIGHT)
    if tile[1] == 0: possible_moves.remove(Move.TOP)
    if tile[1] == 4: possible_moves.remove(Move.BOTTOM)

    return possible_moves

tile_moves = {tile: tile_to_moves(tile) for tile in BORDER}

ALL_MOVES = []
for tile in BORDER:
    possible_moves = tile_moves[tile]
    for move in possible_moves: ALL_MOVES.append((tile, move))
N_ALL = len(ALL_MOVES)

class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:

        from_pos = random.choice(BORDER)
        while game.get_board()[from_pos[1], from_pos[0]] == 1 - game.current_player_idx: from_pos = random.choice(BORDER)

        possible_moves = tile_moves[from_pos]
        
        move = random.choice(possible_moves)

        return from_pos, move

16


State could be considered as a single number if considering 0-presence and 1-presence as bits

In [120]:
import numpy as np

def state_to_board(state):
    binary_string = format(state, '051b')
    tmp_array = np.array(list(map(int, binary_string)))

    binary_array = tmp_array[:-1].reshape(2, 5, 5)
    next_to_move = tmp_array[-1]

    board = np.zeros((5, 5), dtype=int)
    board[binary_array[0] == 1] = -1
    board[binary_array[1] == 1] = 1

    return board, next_to_move

def board_to_state(board, next_to_move):
    binary_array = np.zeros((2, 5, 5), dtype=int)
    
    binary_array[0][board == -1] = 1
    binary_array[1][board == 1] = 1

    binary_string = ''.join(map(str, list(binary_array.flatten()) + [next_to_move]))
    return int(binary_string, 2)



rand_board = np.random.choice([-1, 0, 1], size=(5, 5), replace=True)
print('starting Board:')
print(rand_board)
print(1)

rand_state = board_to_state(rand_board, 1)

print('\nState:')
print(rand_state)

print('\nreconsntructed Board:')
rec_board, next_to_move = state_to_board(rand_state)
print(rec_board)
print(next_to_move)

starting Board:
[[ 0  1  0  0  1]
 [ 0  0  1 -1  0]
 [ 0 -1  0  0  1]
 [-1  0 -1  0  1]
 [ 0  1 -1 -1 -1]]
1

State:
4991240898641

reconsntructed Board:
[[ 0  1  0  0  1]
 [ 0  0  1 -1  0]
 [ 0 -1  0  0  1]
 [-1  0 -1  0  1]
 [ 0  1 -1 -1 -1]]
1


there are 8 simmetries total (including no simmetry) - methods to check simmetry and to rotate (from_pos, move)

In [121]:
dict_rot = {
    (Move.TOP, 1): Move.RIGHT,
    (Move.TOP, 2): Move.BOTTOM,
    (Move.TOP, 3): Move.LEFT,
    (Move.BOTTOM, 1): Move.LEFT,
    (Move.BOTTOM, 2): Move.TOP,
    (Move.BOTTOM, 3): Move.RIGHT,
    (Move.LEFT, 1): Move.TOP,
    (Move.LEFT, 2): Move.RIGHT,
    (Move.LEFT, 3): Move.BOTTOM,
    (Move.RIGHT, 1): Move.BOTTOM,
    (Move.RIGHT, 2): Move.LEFT,
    (Move.RIGHT, 3): Move.TOP,
}

dict_flip = {
    Move.TOP: Move.TOP,
    Move.BOTTOM: Move.BOTTOM,
    Move.LEFT: Move.RIGHT,
    Move.RIGHT: Move.LEFT,
}

#rot_orario: (3, 4) -> (4, 1) -> (1, 0) -> (0, 3) -> (3, 4)
#: (xi, yi) -> (yi, 4 - xi)
#rot_anti_orario: (3, 4) -> (0, 3) -> (1, 0) -> (4, 1) -> (3, 4)
#: (xi, yi) -> (4 - yi, xi)

def rot(n_rot):
    def rot_n(from_pos, move):
        for _ in range(n_rot):
            from_pos = 4 - from_pos[1], from_pos[0]
        return from_pos, dict_rot[(move, n_rot)]
    return rot_n

def flip(from_pos, move):
    from_pos = 4 - from_pos[0], from_pos[1]
    return from_pos, dict_flip[move]

def flip_rot(n_rot):
    def flip_rot_n(from_pos, move):
        from_pos, move = rot(n_rot)(from_pos, move)
        return flip(from_pos, move)
    return flip_rot_n

rot1 = rot(1)
rot2 = rot(2)
rot3 = rot(3)
flip_rot1 = flip_rot(1)
flip_rot2 = flip_rot(2)
flip_rot3 = flip_rot(3)

verse_simmetries = [
    rot3,
    rot2,
    rot1,
    flip,
    flip_rot3,
    flip_rot2,
    flip_rot1,
]

inverse_simmetries = [
    rot1,
    rot2,
    rot3,
    flip,
    flip_rot1,
    flip_rot2,
    flip_rot3,
]

def check_simmetries(board, next_to_move, state_list):

    base_state = tuple(list(board.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, None

    R1 = np.rot90(board)
    base_state = tuple(list(R1.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 0

    R2 = np.rot90(R1)
    base_state = tuple(list(R2.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 1

    R3 = np.rot90(R2)
    base_state = tuple(list(R3.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 2
    
    F = np.fliplr(board)
    base_state = tuple(list(F.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 3
    
    FR1 = np.rot90(F)
    base_state = tuple(list(FR1.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 4
    
    FR2 = np.rot90(FR1)
    base_state = tuple(list(FR2.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 5
    
    FR3 = np.rot90(FR2)
    base_state = tuple(list(FR3.flatten()) + [next_to_move])
    if base_state in state_list: return base_state, 6
    
    return tuple(list(board.flatten()) + [next_to_move]), None

MOVES_SIMMETRIES = {} #(id_move, id_simmetry) -> id_move

for id_move in range(len(ALL_MOVES)):
    from_pos, move = ALL_MOVES[id_move]

    for id_simmetry in range(len(inverse_simmetries)):

        idx = None
        for i in range(len(ALL_MOVES)):
            if ALL_MOVES[i] == inverse_simmetries[id_simmetry](from_pos, move):
                idx = i
                break
        
        MOVES_SIMMETRIES[(id_move, id_simmetry)] = i

print(len(MOVES_SIMMETRIES))
print(len(ALL_MOVES) * 7)
c = 10
for k, v in MOVES_SIMMETRIES.items():
    print((k, v))
    c -= 1
    if c == 0: break

308
308
((0, 0), 31)
((0, 1), 18)
((0, 2), 27)
((0, 3), 41)
((0, 4), 39)
((0, 5), 21)
((0, 6), 14)
((1, 0), 30)
((1, 1), 17)
((1, 2), 28)


time class to time which state give the best performance

In [122]:
import time

class TimeCounter:
    def __init__(self):
        self.tot_time = 0
        self.count = 0
    
    def add_t(self, t):
        self.tot_time += t
        self.count += 1

    def get(self): return self.tot_time, (self.tot_time / self.count) if self.count > 0 else 0

Minimax class

In [123]:
def evaluate_board(board, winner, maximizing_player):

    win_reward = 1

    base_reward = -1 * win_reward # winner == 1 - maximizing_player
    if winner == maximizing_player: base_reward = win_reward
    elif winner == -1: base_reward = 0

    return base_reward

def minimax(game: "Dummy_Game", board, depth, maximizing, current_player, alpha, beta, transposition_table, time_counter):

    time_start = time.perf_counter()

    ##################

    #state = tuple(list(board.flatten()) + [current_player])
    #if state in transposition_table: return transposition_table[state]

    #

    #state = board_to_state(board, current_player)
    #if state in transposition_table: return transposition_table[state]

    #

    state, _ = check_simmetries(board, current_player, transposition_table)

    ##################
    
    time_counter.add_t((time.perf_counter() - time_start) * 1e6)

    winner = game.check_winner()
    if winner != -1 or depth == 0: return evaluate_board(board, winner, current_player if maximizing else 1 - current_player)

    if maximizing:
        max_eval = float('-inf')
        for om in ALL_MOVES:
            from_pos, move = om
            new_board, ok = game.single_move(board, from_pos, move, current_player)
            if ok:
                eval = minimax(game, new_board, depth - 1, False, 1 - current_player, alpha, beta, transposition_table, time_counter)

                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)

                if beta <= alpha: break  # Prune the remaining branches

        transposition_table[state] = max_eval
        return max_eval
    
    else:
        min_eval = float('inf')
        for om in ALL_MOVES:
            from_pos, move = om
            new_board, ok = game.single_move(board, from_pos, move, current_player)
            if ok:
                eval = minimax(game, new_board, depth - 1, True, 1 - current_player, alpha, beta, transposition_table, time_counter)

                min_eval = min(min_eval, eval)
                beta = min(beta, eval)

                if beta <= alpha: break  # Prune the remaining branches

        transposition_table[state] = min_eval
        return min_eval

def minimax_wrapper(game: "Dummy_Game", board, player_id, max_depth, transposition_table= {}):

    best_move = None
    best_eval = float('-inf')

    time_counter = TimeCounter()

    for om in ALL_MOVES:
        from_pos, move = om
        new_board, ok = game.single_move(board, from_pos, move, player_id)
        if ok:
            om_eval = minimax(game, new_board, max_depth, False, 1 - player_id, float('-inf'), float('inf'), transposition_table, time_counter)

            if om_eval > best_eval:
                best_eval = om_eval
                best_move = om

    #print(f"Best move: {best_move} -> Best Value: {best_eval}")
    
    tt, avgt = time_counter.get()
    #print(f"Elapsed time: {(tt, avgt)} microseconds")
    #print(f"Elapsed time: {tt / 1e6, avgt / 1e6} seconds")

    return best_move, tt, avgt

initial_board = np.ones((5, 5)) * -1
initial_board[1:, 0] = 1

minimax_wrapper(Dummy_Game(), initial_board, 0, 2)

(((4, 4), <Move.LEFT: 2>), 238200.40188729763, 63.30066486508042)

In [124]:
class MyPlayer(Player):
    def __init__(self, max_depth= 0, base_until_move_change= 10) -> None:
        super().__init__()

        self.max_depth = max_depth
        self.transposition_table = {}

        self.dummy = Dummy_Game()

        self.base_until_move_change = base_until_move_change
        self.last_pos_move = None
        self.until_move_change = self.base_until_move_change

        self.tt = 0
        self.avgt = 0

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
    
        board = game.get_board()

        pos_move, tt, avgt = minimax_wrapper(self.dummy, board, game.current_player_idx, max_depth= self.max_depth, transposition_table= self.transposition_table)
        
        self.tt += tt
        self.avgt += avgt

        if pos_move == self.last_pos_move:
            self.until_move_change -= 1
            if self.until_move_change == 0:
                board = game.get_board()
                player_id = game.current_player_idx

                from_pos, move = ALL_MOVES[np.random.randint(0, N_ALL)]
                while board[from_pos[1], from_pos[0]] == 1 - player_id: from_pos, move = ALL_MOVES[np.random.randint(0, N_ALL)]

                self.until_move_change = self.base_until_move_change
        else: self.last_pos_move = pos_move

        from_pos, move = pos_move
        
        #print((from_pos, move))
        #print('---------------')
        return from_pos, move

train and test

In [125]:
ThePlayer = MyPlayer(0)

wins_first = 0
wins_second = 0
n_trials = 100

for i_g in tqdm(range(n_trials)):
#for i_g in range(n_trials):

    #print('==============================================================================')
    #print('==============================================================================')
    #print(f'game {i_g+1} - Player First (0)')

    game = Game()
    winner = game.play(ThePlayer, RandomPlayer())
    if winner == 0: wins_first += 1

    #print(f'game {i_g+1} - Player Second (1)')

    game = Game()
    winner = game.play(RandomPlayer(), ThePlayer)
    if winner == 1: wins_second += 1

print(f"Player won {wins_first} / {n_trials} as first")
print(f"Player won {wins_second} / {n_trials} as second")

print(f"Elapsed time: {(ThePlayer.avgt)} microseconds")

100%|██████████| 100/100 [00:10<00:00,  9.12it/s]

Player won 74 / 100 as first
Player won 73 / 100 as second
Elapsed time: 202000.58662929686 microseconds


In [126]:
ThePlayer = MyPlayer(1)

wins_first = 0
wins_second = 0
n_trials = 100

for i_g in tqdm(range(n_trials)):
#for i_g in range(n_trials):

    #print('==============================================================================')
    #print('==============================================================================')
    #print(f'game {i_g+1} - Player First (0)')

    game = Game()
    winner = game.play(ThePlayer, RandomPlayer())
    if winner == 0: wins_first += 1

    #print(f'game {i_g+1} - Player Second (1)')

    game = Game()
    winner = game.play(RandomPlayer(), ThePlayer)
    if winner == 1: wins_second += 1

print(f"Player won {wins_first} / {n_trials} as first")
print(f"Player won {wins_second} / {n_trials} as second")

print(f"Elapsed time: {(ThePlayer.avgt)} microseconds")

100%|██████████| 100/100 [07:14<00:00,  4.34s/it]

Player won 93 / 100 as first
Player won 94 / 100 as second
Elapsed time: 264771.1744706074 microseconds


In [127]:
ThePlayer = MyPlayer(2)

wins_first = 0
wins_second = 0
n_trials = 100

for i_g in tqdm(range(n_trials)):
#for i_g in range(n_trials):

    #print('==============================================================================')
    #print('==============================================================================')
    #print(f'game {i_g+1} - Player First (0)')

    game = Game()
    winner = game.play(ThePlayer, RandomPlayer())
    if winner == 0: wins_first += 1

    #print(f'game {i_g+1} - Player Second (1)')

    game = Game()
    winner = game.play(RandomPlayer(), ThePlayer)
    if winner == 1: wins_second += 1

print(f"Player won {wins_first} / {n_trials} as first")
print(f"Player won {wins_second} / {n_trials} as second")

print(f"Elapsed time: {(ThePlayer.avgt)} microseconds")

  0%|          | 0/100 [00:00<?, ?it/s]

 46%|████▌     | 46/100 [09:23<11:00, 12.24s/it]


KeyboardInterrupt: 